In [16]:
# The goal is to find the set of all moduli q for which B_q, as defined by (33) in the paper, is less than 2*C(q,1),
# where C(q,1) is the number of square roots of 1 mod q.

from sage.libs.lcalc.lcalc_Lfunction import Lfunction_from_character
 
# For a given primitive character chi mod q, the two functions below give the upper and lower bounds for the tail 
# of the defining series of B_{chi} truncated at the Nth zero. (Here B_{chi}=sum_{gamma} (1/4+gamma^2)^{-1}.)

def TailUpperBound2(q, N, chi):
    L = Lfunction_from_character(chi);
    # if chi is real, L.find_zeros_via_N returns only the positive zeros, so we include half of them to account for
    # the symmetric, negative zeros.
    if chi.multiplicative_order() == 2:
        NextZero = abs(L.find_zeros_via_N(N//2+1)[N//2])
        if NextZero > 0.57:
            Integral = numerical_integral(2*x*(x*log(abs(q)*x/(2*pi*e))/pi + 0.25 + 0.22737*log(abs(q)*(x+2)/(2*pi)) 
                                               + 2*log(1+log(abs(q)*(x+2)/(2*pi))) - 0.5)/(0.25+ x^2)^2, NextZero, +Infinity);
            return -N/(0.25+NextZero^2) + Integral[0]    
    else:
        NextZero = abs(L.find_zeros_via_N(N+1)[N])
        if NextZero > 0.57:
            Integral = numerical_integral(x*(x*log(abs(q)*x/(2*pi*e))/pi + 0.25 + 0.22737*log(abs(q)*(x+2)/(2*pi)) 
                                             + 2*log(1+log(abs(q)*(x+2)/(2*pi))) - 0.5)/(0.25+ x^2)^2, NextZero, +Infinity);
            return -N/(0.25+NextZero^2) + Integral[0]


def TailLowerBound2(q, N, chi):
    L = Lfunction_from_character(chi);
    if chi.multiplicative_order() == 2:
        NextZero = abs(L.find_zeros_via_N(N//2+1)[N//2])
        if NextZero > 0.57:
            Integral = numerical_integral(2*x*(x*log(abs(q)*x/(2*pi*e))/pi - 0.25 - 0.22737*log(abs(q)*(x+2)/(2*pi)) 
                                               - 2*log(1+log(abs(q)*(x+2)/(2*pi))) + 0.5)/(0.25+ x^2)^2, NextZero, +Infinity);
            return -N/(0.25 + NextZero^2) + Integral[0]
    else:
        NextZero=abs(L.find_zeros_via_N(N+1)[N])
        if NextZero > 0.57:
            Integral = 2*numerical_integral(2*x*(x*log(abs(q)*x/(2*pi*e))/pi - 0.25 - 0.22737*log(abs(q)*(x+2)/(2*pi)) 
                                                 - 2*log(1+log(abs(q)*(x+2)/(2*pi))) + 0.5)/(0.25+ x^2)^2, NextZero, +Infinity);
            return -N/(0.25 + NextZero^2) + Integral[0]

In [39]:
# The following functions computes C(q,1) for a given q.
def C(q):
    factors = list(factor(q));
    if is_odd(q) or factors[0][1] == 2:
        return 2^(len(factors));
    elif factors[0][1] >= 3:
        return 2^(len(factors) + 1);
    else:
        return 2^(len(factors) - 1);

def square(List):
    Sum = 0;
    for i in List: 
        Sum = Sum + 1/(0.25 + i^2);
    return Sum

# Given a list of candidates of moduli q, the following function outputs those "true" q's for which we must have 
# B_{q} < 2*C(q,1) by calculating the partial sum over the first N zeros and estimating the tail by the previous 
# two functions. If N is not large enough to tell whether B_{q} < 2*C(q,1), it outputs "not sure."

def check(N, candidates):
    true_list = [];
    not_sure_list = [];
    for q in candidates:
        i = 1;
        SumUpper = 0.046191; # The sum for the principal character mod q
        SumLower = 0.046191;
        n = euler_phi(q) - 1; # The number of non-principal characters mod q
        C_q = C(q);
        while i <= n:
            chi = DirichletGroup(q)[i].primitive_character(); # The primitive character inducing the i^th character.
            c = chi.conductor();
            TailUpper = TailUpperBound2(c, N, chi);
            TailLower = TailLowerBound2(c, N, chi);
            L = Lfunction_from_character(chi);
            if chi.multiplicative_order() == 2: # if chi is real
                SumUpper += 2 * square(L.find_zeros_via_N(N//2)) + TailUpper # Max possible value of B_chi
                SumLower += 2 * square(L.find_zeros_via_N(N//2)) - TailLower # Min possible value of B_chi
            else:
                SumUpper += square(L.find_zeros_via_N(N)) + TailUpper
                SumLower += square(L.find_zeros_via_N(N)) - TailLower
            if SumLower > 2 * C_q:
                break
            i += 1;           
        if SumUpper < 2 * C_q:
            true_list.append(q);
        elif SumLower < 2 * C_q and SumUpper >= 2 * C_q:    
            not_sure_list.append(q)          
      
    print("true:", true_list, "notsure:", not_sure_list)

In [50]:
# This function calculates n(q), the number of primitive characters mod q
def n(q):
    number = q;
    factors = list(factor(q));
    for p in factors:
        if p[1] == 1:
            number *= 1 - 2/p[0];
        else:
            number *= (1 - 1/p[0])^2;
    return number   

In [52]:
# Computing the list of candidates of q mentioned in section 7.
candidates = [];
for q in range(5, 2041):
    integral = numerical_integral(2*x*(x*log(abs(q)*x/(2*pi*e))/pi - 0.25 - 0.22737*log(abs(q)*(x+2)/(2*pi)) 
                                       - 2*log(1+log(abs(q)*(x+2)/(2*pi))) + 0.5)/(0.25+ x^2)^2, 4, +Infinity);
    if mod(q,4) != 2 and n(q) * integral[0] <= 2 * C(q):
        candidates.append(q);

check(250, candidates)

true: [5, 7, 8, 9, 12, 15, 16, 20, 21, 24, 28, 36, 40, 48, 60] notsure: [120]


In [59]:
check(700, [120])

true: [] notsure: []


In [ ]:
# To obtain the complete list of Q in Theorem 4, we also have to add q = 2, 3, 4 and those q that are 2 mod 4 
# such that q/2 belongs to the list output above.